# Objectif : trouver la question du thème la plus similaire

Le but de ce code est de déterminer, pour une entrée utilisateur et un thème donné, la question du thème la plus similaire, afin de renvoyer la réponse associée.

In [1]:
import pandas as pd
import spacy
import numpy as np

nlp = spacy.load("fr_core_news_sm")

In [2]:
faq = pd.read_pickle('faq_centerPark.pkl')

## 1. Nettoyage du corpus

Nous allons procéder à un nettoyage du tableau de données. Nous définissons un certain nombre de fonctions qui nous permettrons de nettoyer les questions.

In [3]:
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('french')

sw = nltk.corpus.stopwords.words('french')
sw += ['être', 'avoir']
sw.sort()

def lemmatise_text(text):
    lst_lematised = [token.lemma_ for token in nlp(text)] 
    return ' '.join(lst_lematised).lower()


def stem_text(text):
    lst_stemmerised = [stemmer.stem(token) for token in word_tokenize(text)]    
    return ' '.join(lst_stemmerised)


def substitute_punctuation(text):
    return ' '.join(text.replace("'", ' ').translate(str.maketrans('', '', string.punctuation)).split())


def supp(text):
    return text.replace("«", "").replace("’", "").replace("•", "").replace("®", "")


def supp_sw(text):
    return ' '.join([token.text for token in nlp(text) if not token.text in sw])

In [4]:
faq['question_clean'] = (faq.question.apply(lemmatise_text)
                                     .apply(stem_text)
                                     .apply(substitute_punctuation)
                                     .apply(supp)
                                     .apply(supp_sw)
                        )

Nous sauvegardons ce tableau nettoyé puisque nous nous en servirons directement dans le chatbot.

In [5]:
faq.to_pickle('df_classif_similarity.pkl')

Nous tokenisons les questions grâce au module `spacy`.

In [6]:
faq['tokens'] = faq['question_clean'].apply(nlp)

## 2. Sélection des questions du thème

Afin de comprendre le fonctionnement de la réponse par similarité, nous fixons un thème et une question utilisateur. En effet, nous supposons que l'utilisateur a saisi sa question, que cette dernière a été classifiée dans le domaine Center Parcs, puis dans un thème. 

In [7]:
theme_quest_user = "Préparer mon séjour"

In [19]:
quest_user = "Puis-je évaluer le confort de mon domaine ?"
#quest_user = "Les annimaux domestiques sont-ils acceptés ?"
#quest_user = "ai-je besoin d'un visa ou d'un passeport ?"

In [20]:
faq_theme = faq[faq.theme == theme_quest_user]

## 3. Nettoyage de la question

Nous appliquons le même traitement à la question de l'utilisateur.

In [21]:
quest_user_clean = supp_sw(supp(substitute_punctuation(stem_text(lemmatise_text(quest_user)))))

## 4. Recherche de la question de la FAQ la plus similaire

In [22]:
quest_user_clean_tokens = nlp(quest_user_clean)

In [23]:
lst_similarity = [quest_user_clean_tokens.similarity(token) for token in faq_theme.tokens]

C:\Users\enora\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
C:\Users\enora\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larg

In [24]:
rep_quest_user = faq_theme.reponse[np.asarray(lst_similarity).argmax()]

In [25]:
rep_quest_user

"Le classement par « birdies » évalue l’offre Center Parcs dans son ensemble : l’environnement nature, l’Aqua Mundo, le Dôme, les Activités et animations pour les enfants, le spa, etc. Les parcs sont classés de 3 à 5 birdies. En France, tous les domaines sont « 5 birdies » (note maximale).La classification officielle Résidences de Tourisme Arrêté de 2010  - les fameuses étoiles – évalue le niveau de confort et de prestation uniquement sur l’hébergement, donc sur les cottages de Center Parcs. - Le Bois aux Daims : 4 étoiles- Les Trois Forêts : 4 étoiles- Le Lac d'Ailette : 3 étoiles- Les Bois-Francs : 3 étoiles- Les Hauts de Bruyères : 3 étoiles"

## 5. Evaluation de la performance

Afin d'évaluer de manière quantitative le modèle sélectionné, nous avons créé un jeu de questions - réponses dont les réponses sont mot pour mot celles de la FAQ mais dont les questions ont été imaginées par nos soins. L'objectif étant de comparer la réponse retournée à celle attendue.

Nous allons procéder thème par thème.

In [45]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

In [27]:
df_test_quanti = pd.read_csv('jeu_test_similarite.csv', ";")
df_test_quanti.head()

,question,reponse,theme
0,Quels sont les jours d’arrivée ?,Il est possible d’arriver n’importe quel jour ...,Préparer mon séjour
1,Comment évaluer le confort de mon domaine et d...,Le classement par « birdies » évalue l’offre C...,Préparer mon séjour
2,Quels sont les services et activités compris d...,"En réservant votre hébergement, vous bénéficie...",Préparer mon séjour
3,Comment réserver mes activités ?,"Lors de la réservation de votre hébergement, v...",Préparer mon séjour
4,Où trouver le plan du domaine ?,"Sur la page d'accueil de notre site, cliquez s...",Préparer mon séjour


In [44]:
df_test_quanti['question_clean'] = (df_test_quanti.question.apply(lemmatise_text)
                                     .apply(stem_text)
                                     .apply(substitute_punctuation)
                                     .apply(supp)
                                     .apply(supp_sw)
                        )
df_test_quanti['tokens'] = df_test_quanti['question_clean'].apply(nlp)
df_test_quanti['reponse_pred'] = ''

for theme in set(df_test_quanti.theme):
    faq_theme = faq[faq.theme == theme]
    trouve_similarity = lambda t: faq_theme.iloc[np.asarray([t.similarity(token) for token in faq_theme.tokens]).argmax()].reponse
    df_test_quanti[df_test_quanti.theme == theme]['reponse_pred'] = df_test_quanti[df_test_quanti.theme == theme_quest_user].tokens.apply(trouve_similarity)

C:\Users\enora\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
C:\Users\enora\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larg

C:\Users\enora\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
C:\Users\enora\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larg

C:\Users\enora\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
C:\Users\enora\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larg

C:\Users\enora\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
C:\Users\enora\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larg

C:\Users\enora\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
C:\Users\enora\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larg

...

C:\Users\enora\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
C:\Users\enora\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larg

C:\Users\enora\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
C:\Users\enora\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larg

C:\Users\enora\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
C:\Users\enora\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larg

C:\Users\enora\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
C:\Users\enora\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larg

C:\Users\enora\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
C:\Users\enora\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larg

In [46]:
df_test_quanti

,question,reponse,theme,question_clean,tokens,reponse_pred
0,Quels sont les jours d’arrivée ?,Il est possible d’arriver n’importe quel jour ...,Préparer mon séjour,quel jour arriv,"(quel, jour, , arriv)",
1,Comment évaluer le confort de mon domaine et d...,Le classement par « birdies » évalue l’offre C...,Préparer mon séjour,comment évalu confort domain cottag,"(comment, évalu, confort, domain, cottag)",
2,Quels sont les services et activités compris d...,"En réservant votre hébergement, vous bénéficie...",Préparer mon séjour,quel servic activ compr dan prix cottag,"(quel, servic, activ, compr, dan, prix, cottag)",
3,Comment réserver mes activités ?,"Lors de la réservation de votre hébergement, v...",Préparer mon séjour,comment réserv activ,"(comment, réserv, activ)",
4,Où trouver le plan du domaine ?,"Sur la page d'accueil de notre site, cliquez s...",Préparer mon séjour,où trouv plan domain,"(où, trouv, plan, domain)",
5,Puis-je venir avec mon animal domestique ?,La présence de votre animal domestique est aut...,Préparer mon séjour,puis ven animal domest,"(puis, ven, animal, domest)",
6,Peut-on accéder au domaine à la journée (sans ...,Il est possible de passer la journée à Center ...,Préparer mon séjour,pouvoir acced domain journ san réserv cottag,"(pouvoir, acced, domain, journ, san, réserv, c...",
7,Comment réserver un logement adapté aux person...,"Tous nos domaines français, dont Villages Nat...",Préparer mon séjour,comment réserv log adapt person mobil réduir,"(comment, réserv, log, adapt, person, mobil, r...",
8,Comment recevoir la brochure ?,Cliquez ici pour remplir une demande d'envoi d...,Préparer mon séjour,comment recevoir brochur,"(comment, recevoir, brochur)",
9,Est-il possible d'acheter des billets pour les...,Oui. Achetez directement vos billets lors de v...,Préparer mon séjour,possibl achet billet parc disneyland paris,"(possibl, achet, billet, parc, disneyland, ,...",


In [52]:
df_test_quanti[df_test_quanti.theme == 'Préparer mon séjour']['reponse_pred'] = 'a'

C:\Users\enora\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [48]:
df_test_quanti

,question,reponse,theme,question_clean,tokens,reponse_pred
0,Quels sont les jours d’arrivée ?,Il est possible d’arriver n’importe quel jour ...,Préparer mon séjour,quel jour arriv,"(quel, jour, , arriv)",
1,Comment évaluer le confort de mon domaine et d...,Le classement par « birdies » évalue l’offre C...,Préparer mon séjour,comment évalu confort domain cottag,"(comment, évalu, confort, domain, cottag)",
2,Quels sont les services et activités compris d...,"En réservant votre hébergement, vous bénéficie...",Préparer mon séjour,quel servic activ compr dan prix cottag,"(quel, servic, activ, compr, dan, prix, cottag)",
3,Comment réserver mes activités ?,"Lors de la réservation de votre hébergement, v...",Préparer mon séjour,comment réserv activ,"(comment, réserv, activ)",
4,Où trouver le plan du domaine ?,"Sur la page d'accueil de notre site, cliquez s...",Préparer mon séjour,où trouv plan domain,"(où, trouv, plan, domain)",
5,Puis-je venir avec mon animal domestique ?,La présence de votre animal domestique est aut...,Préparer mon séjour,puis ven animal domest,"(puis, ven, animal, domest)",
6,Peut-on accéder au domaine à la journée (sans ...,Il est possible de passer la journée à Center ...,Préparer mon séjour,pouvoir acced domain journ san réserv cottag,"(pouvoir, acced, domain, journ, san, réserv, c...",
7,Comment réserver un logement adapté aux person...,"Tous nos domaines français, dont Villages Nat...",Préparer mon séjour,comment réserv log adapt person mobil réduir,"(comment, réserv, log, adapt, person, mobil, r...",
8,Comment recevoir la brochure ?,Cliquez ici pour remplir une demande d'envoi d...,Préparer mon séjour,comment recevoir brochur,"(comment, recevoir, brochur)",
9,Est-il possible d'acheter des billets pour les...,Oui. Achetez directement vos billets lors de v...,Préparer mon séjour,possibl achet billet parc disneyland paris,"(possibl, achet, billet, parc, disneyland, ,...",
